## First, loading the data and defining the functions

In [45]:
import pandas as pd
df_letters = pd.read_csv('data/all_letters.tsv',sep='\t')

In [46]:
# manually adding historical spelling variations for opium and spices we're interested in
# from https://gtb.ivdnt.org/search/

historical_labels_per_commodity = {'kaneel': ['canella','caneel','canele','caneele',
                               'kanele','canelle'],
                    'nootmuskaat':['notemosscate', 'notemetscate', 'notemisscate','musschaeten',
                                  'notemusscate','note musscate','muscaetnot','muskaatnoot',
                                  'muskaat','musscat','muscaet','metscat','misscat','mosscat'],
                    'foelie':['folye','folie'],
                    'opium':['amphioen','amfioen'],
                    'peper':[]}

In [47]:
import json

In [48]:
global matching_mode
'''
options:
        "exact": "kaneel" and "kaneelstokjes" won't match (minimise false positives)
        "broad": "gin" and "beginnen" will match (minimise false negatives)
        "left_only": "kaneel" and "kaneelstokjes" match but "gin" and "beginnen" don't (a middle ground)
'''


'\noptions:\n        "exact": "kaneel" and "kaneelstokjes" won\'t match (minimise false positives)\n        "broad": "gin" and "beginnen" will match (minimise false negatives)\n        "left_only": "kaneel" and "kaneelstokjes" match but "gin" and "beginnen" don\'t (a middle ground)\n'

In [49]:
import re

def find_label(text, label):
    if matching_mode == 'left_only':
        pattern = r'(?<![a-zA-Z])' + re.escape(label)
    if matching_mode == 'exact':
        pattern = r'(?<![a-zA-Z])' + re.escape(label) + r'(?![a-zA-Z])'
    if matching_mode == 'broad':
        pattern = re.escape(label)
        
    regex = re.compile(pattern)
    
    return re.search(regex, text)

In [50]:
# test
matching_mode = 'exact'
find_label('We beginnen met een beetje gin', 'gin')

<re.Match object; span=(27, 30), match='gin'>

In [51]:
def find_opium_and_spices(text): # only looking for our target commodities
    res = {}
    if type(text) != str:
        return '{}'
    
    
    for main_label, extra_labels in historical_labels_per_commodity.items():
        for label in [main_label]+extra_labels: # searching for all variations
            if find_label(text.lower(), label.lower()): 
                if main_label in res: # different labels for the same uri already found
                    res[main_label].append(label)
                else:
                    res[main_label] = [label]
    return json.dumps(res)

## Looking for opium and target spices

In [52]:
matching_mode = 'left_only'
df_letters['commodity_labels'] = df_letters['text'].apply(find_opium_and_spices)
matching_mode = 'exact'
df_letters['commodity_labels_precise'] = df_letters['text'].apply(find_opium_and_spices)
df_letters.to_csv('data/letters_with_opium_and_spices.tsv',sep='\t',index=False)

In [53]:
df_letters

,page,title,rawdate,seq,place,year,month,day,author,pid,status,authorFull,text,rest,commodity_labels,commodity_labels_precise
0,3,Both; aan boord van het Wapen van Amsterdam vo...,25 februari 1610,I,aan boord van het Wapen van Amsterdam voor Ile...,1610.0,2.0,25.0,Both,NaN,NaN,Pieter Both,"«Scheepshericht, vnl. handelend over uitgedeel...",NaN,{},{}
1,3,Both; aan boord van het Wapen van Amsterdam li...,6 augustus 1610,II,aan boord van het Wapen van Amsterdam liggende...,1610.0,8.0,6.0,Both,NaN,NaN,Pieter Both,"«„Journaelsgewijse"" reisbeschrijving »",NaN,{},{}
2,3,Both; aan boord van het Wapen van Amsterdam te...,14 oktober 1610,III,aan boord van het Wapen van Amsterdam ter Rede...,1610.0,10.0,14.0,Both,NaN,NaN,Pieter Both,«De inhoud van deze brief is evenmin als die v...,NaN,{},{}
3,4,"Both; Kasteel Nassau op Banda-Neira, 18 juli 1611",18 juli 1611,IV,Kasteel Nassau op Banda-Neira,1611.0,7.0,18.0,Both,NaN,NaN,Pieter Both,. . . Principaelijck besich geweest in ’t opbo...,NaN,{},{}
4,7,"Both; aan boord van de Vere voor Maleyo, 31 ma...",31 maart 1612,V,aan boord van de Vere voor Maleyo,1612.0,3.0,31.0,Both,NaN,NaN,Pieter Both,"Mijn advys soude oock sijn, dat U E geen licht...",NaN,{},{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,745,"Van der Parra, Van Riemsdijk, Schreuder, Van B...",31 december 1765,XIV,Batavia,1765.0,12.0,31.0,"Van der Parra, Van Riemsdijk, Schreuder, Van B...",NaN,NaN,"Petrus Albertus van der Parra, Jeremias van Ri...",NaN,NaN,{},{}
603,883,"Van der Parra, Van Riemsdijk, Schreuder, Van B...",8 april 1766,XV,Batavia,1766.0,4.0,8.0,"Van der Parra, Van Riemsdijk, Schreuder, Van B...",NaN,NaN,"Petrus Albertus van der Parra, Jeremias van Ri...",Nadat het tweede gezandschap op Colombo aangek...,NaN,"{""kaneel"": [""kaneel""], ""opium"": [""opium""], ""pe...","{""kaneel"": [""kaneel""], ""opium"": [""opium""], ""pe..."
604,893,"Van der Parra, Van Riemsdijk, Schreuder, Van B...",21 oktober 1766,XVI,Batavia,1766.0,10.0,21.0,"Van der Parra, Van Riemsdijk, Schreuder, Van B...",NaN,NaN,"Petrus Albertus van der Parra, Jeremias van Ri...",NaN,NaN,{},{}
605,943,"Van der Parra, Van Riemsdijk, Schreuder, Van B...",31 december 1766,XVII,Batavia,1766.0,12.0,31.0,"Van der Parra, Van Riemsdijk, Schreuder, Van B...",NaN,NaN,"Petrus Albertus van der Parra, Jeremias van Ri...",NaN,NaN,{},{}
